# Experiment 2
---------------
Test 1: Custom transfer learning model freezing vgg

### Load default libraries and prepare the dataset

In [ ]:
%run ./prepare_dataset.ipynb

In [ ]:
img_width, img_height, img_size, batch_size =  prepareConstants(224 ,224,224,16)
train_data_dir, validation_data_dir = setDirs()
train_generator, validation_generator = prepareDatasets(train_data_dir,validation_data_dir,img_width,img_height,batch_size)
checkData(train_generator,validation_generator)

### Load the VGG16 Pretrained model from Keras
https://keras.io/applications/#vgg16

In [ ]:
# create the base pre-trained model
vgg_conv = applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

### Test 3

Freeze the net and insert custom layers

In [ ]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

Create my new model

In [ ]:
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))

classes=train_generator.num_classes
print("Number of classes:",classes)
model.add(layers.Dense(classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['acc'])

model.summary()

### Train the model

In [ ]:
start_time = time.time()

with tf.device("/device:GPU:0"):
    # Train the model
    history = model.fit_generator(
          train_generator,
          steps_per_epoch=train_generator.samples/train_generator.batch_size ,
          epochs=5,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples/validation_generator.batch_size,
          verbose=1)



print("\nTraining completed")

elapsed_time = time.time() - start_time
print("Time:",elapsed_time) 

In [ ]:
print("Validation Accuracy max:",history.history['val_acc'][-1])
print("Validation Loss max:",history.history['val_loss'][-1])
print("Training Accuracy max:",history.history['acc'][-1])
print("Training Loss max:",history.history['loss'][-1])


Check the performance for the model

In [ ]:
# summarize history for accuracy
plt.plot(history.history['val_acc'])
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Validation','Training'],loc = 'upper left')
plt.savefig('../plots/exp2_p1_acc.png')
plt.show()


# summarize history for loss
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Validation','Training'],loc = 'upper left')
plt.savefig('../plots/exp2_p1_loss.png')
plt.show()


Problema di: forgetting.
All'epoca 1 e 2 ho un salto sulla loss, in questo momento quindi il modello sta overfittando.  
Per superare questo problema è necessario fare: gradual unfreezing.  
Altra tecnica: chain-thaw.  
I primi esempi hanno un errore molto alto, iniziano a fare back propagation e cambia tanto la rappresentazione astratta.

Save the model

In [ ]:
saveKerasModel(model,"vgg16custom")

Check errors for some image predicted by the model

In [ ]:
errors = seeErrors(validation_data_dir,validation_generator,model)